In [2]:
#final update 2024-12-11
import sys
import os
import numpy as np
import scipy.stats as stats
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sb
import random as rd
import matplotlib
import warnings
import tqdm

In [10]:
targetmz_path = ["D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/plastic_map_chemlist/Plastic_Chemical_withRT_CFMID_spectrumdatabase_chemical_annotation_20250404.csv"] #plastic chemical list with predicted/library RT, CRMID, and spectrum ID match record,
plastics_chem = pd.read_csv(targetmz_path[0])
PLASTICMAP = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/plastic_map_chemlist/PLASTICMAP_comTox dashboard.csv")
#check if DATA_SOURCES column exists in PLASTICMAP
plastics_chem = pd.merge(plastics_chem, PLASTICMAP[['DTXSID', 'DATA_SOURCES']], how='left', left_on='DTXSID', right_on='DTXSID')
plastics_chem.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/plastic_map_chemlist/Plastic_Chemical_withRT_CFMID_spectrumdatabase_chemical_annotation_20250404.csv", index=False)

C:\Users\yangj\AppData\Local\Temp\ipykernel_45332\2005536182.py:3: DtypeWarning: Columns (19,20,59) have mixed types. Specify dtype option on import or set low_memory=False.
  PLASTICMAP = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/plastic_map_chemlist/PLASTICMAP_comTox dashboard.csv")


In [ ]:
def process_peak_area_file(input_path: str,
                           output_path: str,
                           adduct_filter: str = "[M+H]+",
                           drop_prefix: str = "BH") -> pd.DataFrame:
    """
    Read a raw alignment TSV, extract peak‐area data, filter by adduct,
    drop unwanted columns, and write to CSV.
    
    Parameters
    ----------
    input_path : str
        Path to the raw TSV (no header, all cols as text).
    output_path : str
        Path where the filtered CSV will be written.
    adduct_filter : str, default "[M+H]+"
        Which adduct string to keep (e.g. "[M+H]+", "[M-H]-").
    drop_prefix : str, default "BH"
        Prefix of any columns to drop (e.g. batch‐correction flags).
    
    Returns
    -------
    pd.DataFrame
        The filtered peak‐area DataFrame.
    """
    # --- 1) Read raw data block ---
    raw = pd.read_csv(
        input_path,
        sep='\t',
        header=None,
        dtype=str,
        keep_default_na=False
    )
    # drop the first 4 rows to get to the peak-area table
    peak_block = raw.iloc[4:].reset_index(drop=True)
    
    # --- 2) Set column names from the first row, then drop it ---
    peak_block.columns = peak_block.iloc[0]
    peak_block = peak_block.iloc[1:].reset_index(drop=True)
    
    # --- 3) Extract and save the adduct list before numeric conversion ---
    adduct_list = peak_block['Adduct type'].astype(str).tolist()
    
    # --- 4) Select desired columns ---
    cols = list(peak_block.columns)
    msms_idx = cols.index('MS/MS spectrum')
    keep_cols = ['Average Rt(min)', 'Average Mz'] + cols[msms_idx+1:-2]
    
    df = (
        peak_block
        .loc[:, keep_cols]
        .apply(pd.to_numeric, errors='coerce')
    )
    
    # --- 5) Add adduct column, filter, and drop it again ---
    df['Adduct'] = adduct_list
    df = df.loc[df['Adduct'] == adduct_filter].drop(columns=['Adduct'])
    
    # --- 6) Drop any columns whose names start with drop_prefix ---
    df = df.loc[:, ~df.columns.str.startswith(drop_prefix)]
    
    # --- 7) Write out to CSV and return ---
    df.to_csv(output_path, index=False)
    return df


# For [M+H]+
df_pos = process_peak_area_file(
    input_path=r"D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/Area_1_2024_11_04_23_46_26.txt",
    output_path=r"D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/raw_peak_area_before_bc_20250429.csv",
    adduct_filter="[M+H]+"
)

# For [M-H]−
df_neg = process_peak_area_file(
    input_path=r"D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/Area_1_2024_11_04_16_24_07.txt",
    output_path=r"D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/raw_peak_area_before_bc_20250429.csv",
    adduct_filter="[M-H]-"
)

In [ ]:
# Load peak area data for positive ion mode after batch correction
import pandas as pd
from tqdm import tqdm

peakarea_path = ["D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/raw_peak_area_before_bc.csv"] #raw peak area before batch correction (40004, 403)
targetmz_path = ["D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/plastic_map_chemlist/Plastic_Chemical_withRT_CFMID_spectrumdatabase_chemical_annotation_20250404.csv"] #plastic chemical list with predicted/library RT, CRMID, and spectrum ID match record,

#load peak area data
pos_peak_area = pd.read_csv(peakarea_path[0])
print(f'dimension of the peak area data: {pos_peak_area.shape}')
# Load target mz data
targetmzdat_pos = pd.read_csv(targetmz_path[0])
targetmzdat_pos['mz'] = targetmzdat_pos['Monoisotopic_Mass_ready'] + 1.007825 #looking at M+H adducts

# Select relevant columns for inquiry
peak_area_inquiry = pos_peak_area[['Average Rt(min)', 'Average Mz'] + [col for col in pos_peak_area.columns if col.startswith('BH')]]

# Function to calculate ppm difference
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

ppm_threshold = 10 #ppm threshold for matching
matched_rows_pos = []
matched_summary = []

# MS1 search and matching
for _, chem_row in tqdm(targetmzdat_pos.iterrows(), total=targetmzdat_pos.shape[0], desc="Matching compounds"):
    mw = chem_row['mz']
    matches = peak_area_inquiry[peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold)]
    if not matches.empty:
        # Append the matches along with the corresponding compound information
        for _, match_row in matches.iterrows():
            matched_rows_pos.append({
                **match_row,
                'Matched Compound': chem_row['SMILES'],
                'PREFERRED_NAME': chem_row.get('PREFERRED_NAME', None),
                'DTXSID_Hits': chem_row.get('DTXSID', None),
                'DTXSID': chem_row.get('DTXSID', None),
                'InChiKey_origin': chem_row.get('INCHIKEY', None),
                'MOLECULAR_FORMULA_original': chem_row.get('MOLECULAR_FORMULA', None),
                'Compound_comment': chem_row.get('QC_NOTES', None),
                'SMILES_STD': chem_row.get('SMILES_ready', None),
                "Monoisotopic_mass_ready": chem_row.get('Monoisotopic_Mass_ready', None),
                "PredictedRT": chem_row.get('PredRT_Retip', None),
                "PredictedRT_lower": chem_row.get('predRT_lower', None),
                "PredictedRT_upper": chem_row.get('predRT_higher', None),
                'RT_library' : chem_row.get('RT_lib', None),
                'RT_polarity' : chem_row.get('Polarity', None),
                'total_spectra_pos': chem_row.get('total_spectra_pos', None),
                'total_pred_spectra_pos': chem_row.get('total_pred_spectra_pos', None),
                'BloodExpo_check': chem_row.get('BloodExpo_check', None),
                'ExposomeExplorer_check': chem_row.get('ExposomeExplorer_check', None),
                'Function': chem_row.get('Function', None),
                'Polymer': chem_row.get('Polymer', None),
                'US_production': chem_row.get('US_production', None),
                'Total_production': chem_row.get('Total_production', None),
                'Industrial_Sector': chem_row.get('Industrial_Sector', None)
            })

# Create a DataFrame for matched MSDIAL result rows
matched_peak_area_df_pos = pd.DataFrame(matched_rows_pos)

# drop rows with duplicated column values at Average Rt(min) and Average Mz, retaining the first occurrence
matched_peak_area_df_pos_uniquemzrt = matched_peak_area_df_pos.drop_duplicates(subset=['Average Rt(min)', 'Average Mz'], keep='first')

#
import datetime
now = datetime.datetime.now()
folder_name = "D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/"
filename = "unique_features_accuratemassmathced_database_forallsamples_pos"
filetime = now.strftime("%Y%m%d_%H%M%S")
matched_peak_area_df_pos_uniquemzrt.to_csv(os.path.join(folder_name, f"{filename}_{filetime}.csv"),index=False)

print(f"Number of unique matched feature from peak area table: {matched_peak_area_df_pos_uniquemzrt.shape[0]}")
matched_peak_area_df_pos_uniquechemical = matched_peak_area_df_pos.drop_duplicates(subset=['InChiKey_origin'], keep='first')
print(f'Number of unique chemical matched: {matched_peak_area_df_pos_uniquechemical.shape[0]}')

# Step 3: Handling technical duplicates and calculating mean values
bh_columns = [col for col in matched_peak_area_df_pos.columns if col.startswith('BH')]
mean_columns = {}
for col in bh_columns:
    base_col = col.split('_')[0]
    if base_col not in mean_columns:
        duplicate_cols = [c for c in bh_columns if c.startswith(base_col)]
        mean_columns[base_col] = matched_peak_area_df_pos[duplicate_cols].mean(axis=1)

# Retain only the columns with calculated mean values
matched_peak_area_df_pos = pd.concat([matched_peak_area_df_pos.drop(columns=bh_columns), pd.DataFrame(mean_columns)], axis=1)

#output the matched peak area data
#rank the feature by rt AND mz
matched_peak_pos = matched_peak_area_df_pos.sort_values(by=['Average Rt(min)', 'Average Mz'])

#filerted features with intensity lower than 5000 
peak_areapos  = matched_peak_pos[[col for col in matched_peak_pos.columns if col.startswith('BH')]]
row_medianspos  = peak_areapos.median(axis=1)
peak_indexpos  = row_medianspos[row_medianspos >= 0].index
filter_resultspos  = matched_peak_pos.loc[peak_indexpos]
print(f'dimension of feature table filtered by intensity: {filter_resultspos.shape}')

# Step 4: Retain columns with median values > 5000, filter non-numeric values, and perform DataFrame transpose
filtered_df = matched_peak_area_df_pos.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
filtered_df = filtered_df.loc[:, (filtered_df.median(axis=0) >= 0)].dropna(how='all', axis=1)  # Retain columns with median value > 5000

# Count the number of retained columns after intensity filtering
num_retained_columns = filtered_df.shape[1] 
print(f"Number of columns retained after intensity filtering: {num_retained_columns}")
print(f'dimension of the filtered data: {filtered_df.shape}')

# Calculate summary statistics for each sample column
summary_stats = {
    'Sum': filtered_df.sum(),
    'Median': filtered_df.median(),
    'Mean': filtered_df.mean(),
    'Max': filtered_df.max(),
    'Min': filtered_df.min(),
    '95th Percentile': filtered_df.quantile(0.95),
    '5th Percentile': filtered_df.quantile(0.05)
}

# Create a DataFrame from the summary statistics
summary_stats_df_pos = pd.DataFrame(summary_stats)
# summary_stats_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/mathced_peaks_summary_forallsamples_pos_woRT.csv")

# Transpose the summary statistics DataFrame
# transposed_summary_df = summary_stats_df_pos.transpose()
# Display the final DataFrame
# print(summary_stats_df_pos)

dimension of the peak area data: (40004, 463)


Matching compounds: 100%|██████████| 5267/5267 [01:23<00:00, 62.83it/s]


Number of unique matched feature from peak area table: 6248
Number of unique chemical matched: 2190
dimension of feature table filtered by intensity: (12058, 225)
Number of columns retained after intensity filtering: 211
dimension of the filtered data: (12058, 211)


In [15]:
#annotate the features according to RT_library and predictedRT
RTmatch_threshold = 0.6#RT match threshold in minutes
#convert RT_library to numeric values
filter_resultspos['RT_library'] = pd.to_numeric(filter_resultspos['RT_library'], errors='coerce')

for iter, row in filter_resultspos.iterrows():
    if (pd.notnull(row['RT_library'])) and ((row['RT_polarity'] == 'both') or (row['RT_polarity'] == 'pos')):
        if (row['Average Rt(min)'] >= row['RT_library']-0.6) and (row['Average Rt(min)'] <= row['RT_library'] + 0.6):
            filter_resultspos.loc[iter, 'annotation_RTMS'] = '2'
        else:
            filter_resultspos.loc[iter, 'annotation_RTMS'] = 'NotMatched'
    elif (row['RT_polarity'] =='na') or (row['RT_polarity'] == 'neg'):
        filter_resultspos.loc[iter, 'annotation_RTMS'] = 'NotMatched'
    else:
        if (row['Average Rt(min)'] >= row['PredictedRT_lower']) & (row['Average Rt(min)'] <= row['PredictedRT_upper']):
            filter_resultspos.loc[iter, 'annotation_RTMS'] = '3'
        else:
            filter_resultspos.loc[iter, 'annotation_RTMS'] = '4'

print(filter_resultspos[filter_resultspos['annotation_RTMS'] == '2'].shape)
print(filter_resultspos[filter_resultspos['annotation_RTMS'] == '3'].shape)

#output the datafile for spectrum matching
# now = datetime.datetime.now()
# folder_name = "D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/"
# filename = "feature_annotation_RTMS_allfeatures_pos"
# filetime = now.strftime("%Y%m%d_%H%M%S")
# filter_resultspos.to_csv(os.path.join(folder_name, f"{filename}_{filetime}.csv"),index=False)

(53, 226)
(7563, 226)


In [ ]:
# Importing negative alignment results
# Negative alignment results
peakarea_path = ["D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/raw_peak_area_before_bc.csv"] #raw peak area before batch correction
targetmz_path = ["D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/plastic_map_chemlist/Plastic_Chemical_withRT_CFMID_spectrumdatabase_chemical_annotation_20250404.csv"] #plastic chemical list with predicted/library RT, CRMID

#load peak area data
neg_peak_area = pd.read_csv(peakarea_path[0])
# print(f'dimention of the peak area data: {pos_peak_area_200.shape}')
# Load target mz data
targetmzdat_neg = pd.read_csv(targetmz_path[0])
targetmzdat_neg['mz'] = targetmzdat_neg['MONOISOTOPIC_MASS'] - 1.007825

# Select relevant columns for inquiry
peak_area_inquiry = neg_peak_area[['Average Rt(min)', 'Average Mz'] + [col for col in neg_peak_area.columns if col.startswith('BH')]]

# Function to calculate ppm difference
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

ppm_threshold = 10
matched_rows_neg = []
matched_summary = []

# MS1 search and matching
for _, chem_row in tqdm(targetmzdat_neg.iterrows(), total=targetmzdat_neg.shape[0], desc="Matching compounds"):
    mw = chem_row['mz']
    matches = peak_area_inquiry[peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold)]
    #print(f"Matches after ppm filtering: {matches.shape}")  # Debugging
    
    if not matches.empty:
        # Append the matches along with the corresponding compound information
        for _, match_row in matches.iterrows():
            matched_rows_neg.append({
                **match_row,
                'Matched Compound': chem_row['SMILES'],
                'PREFERRED_NAME': chem_row.get('PREFERRED_NAME', None),
                'DTXSID_Hits': chem_row.get('DTXSID', None),
                'DTXSID': chem_row.get('DTXSID', None),
                'InChiKey_origin': chem_row.get('INCHIKEY', None),
                'MOLECULAR_FORMULA_original': chem_row.get('MOLECULAR_FORMULA', None),
                'Compound_comment': chem_row.get('QC_NOTES', None),
                'SMILES_STD': chem_row.get('SMILES_ready', None),
                "Monoisotopic_mass_ready": chem_row.get('Monoisotopic_Mass_ready', None),
                "PredictedRT": chem_row.get('PredRT_Retip', None),
                "PredictedRT_lower": chem_row.get('predRT_lower', None),
                "PredictedRT_upper": chem_row.get('predRT_higher', None),
                'RT_library' : chem_row.get('RT_lib', None),
                'RT_polarity' : chem_row.get('Polarity', None),
                'total_spectra_neg': chem_row.get('total_spectra_neg', None),
                'total_pred_spectra_neg': chem_row.get('total_pred_spectra_neg', None),
                'BloodExpo_check': chem_row.get('BloodExpo_check', None),
                'ExposomeExplorer_check': chem_row.get('ExposomeExplorer_check', None),
                'Function': chem_row.get('Function', None),
                'Polymer': chem_row.get('Polymer', None),
                'US_production': chem_row.get('US_production', None),
                'Total_production': chem_row.get('Total_production', None),
                'Industrial_Sector': chem_row.get('Industrial_Sector', None)
            })
        # Record summary statistics for each targeted mz, preserving multiple hits
        matched_summary.append({
            'Targeted MZ': mw,
            'Matched Count': len(matches),
            'Average Rt(min)': matches['Average Rt(min)'].tolist(),
            'Average Mz': matches['Average Mz'].tolist(),
            'Mean Value': matches.iloc[:, 3:].mean(axis=1).tolist(),
            'Max Value': matches.iloc[:, 3:].max(axis=1).tolist(),
            'Min Value': matches.iloc[:, 3:].min(axis=1).tolist(),
            'Std Dev': matches.iloc[:, 3:].std(axis=1).tolist(),
            'RSD (%)': ((matches.iloc[:, 3:].std(axis=1) / matches.iloc[:, 3:].mean(axis=1)) * 100).tolist(),
            'SMILES_STD': chem_row.get('SMILES_ready', None),
            'DTXSID': chem_row.get('DTXSID', None),
            'CASRN': chem_row.get('CASRN', None),
            'Compound Name': chem_row.get('PREFERRED_NAME', None),
            'QC_NOTES': chem_row.get('QC_NOTES', None),
            'InChiKey_origin': chem_row.get('INCHIKEY', None)
        })

# Create a DataFrame for matched MSDIAL result rows
matched_peak_area_df_neg = pd.DataFrame(matched_rows_neg)

# drop rows with duplicated column values at Average Rt(min) and Average Mz, retaining the first occurrence
matched_peak_area_df_neg_uniquemzrt = matched_peak_area_df_neg.drop_duplicates(subset=['Average Rt(min)', 'Average Mz'], keep='first')
# matched_peak_area_df_neg_uniquemzrt.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/unique_features_accuratemassmathced_database_forallsamples_neg.csv")
matched_peak_area_df_neg_uniquechemical = matched_peak_area_df_neg.drop_duplicates(subset=['InChiKey_origin'], keep='first')
print(f"Number of matched unique feature from peak area table: {matched_peak_area_df_neg_uniquemzrt.shape[0]}")
print(f"Number of matched unique chemicals from peak area table: {matched_peak_area_df_neg_uniquechemical.shape[0]}")

# Create a DataFrame for matched summary statistics
matched_summary_df_neg = pd.DataFrame(matched_summary)
print(f"#of targetedmz with hits:{matched_summary_df_neg.shape}")
# matched_summary_df_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/mathced_targetedmz_summary_forallsamples_Neg_woRT.csv")
# print(matched_summary_df_neg.shape)

# Step 3: Handling technical duplicates and calculating mean values
bh_columns = [col for col in matched_peak_area_df_neg.columns if col.startswith('BH')]
mean_columns = {}
for col in bh_columns:
    base_col = col.split('_')[0]
    if base_col not in mean_columns:
        duplicate_cols = [c for c in bh_columns if c.startswith(base_col)]
        mean_columns[base_col] = matched_peak_area_df_neg[duplicate_cols].mean(axis=1)

# Retain only the columns with calculated mean values
matched_peak_area_df_neg = pd.concat([matched_peak_area_df_neg.drop(columns=bh_columns), pd.DataFrame(mean_columns)], axis=1)

#output the matched peak area data
#rank the feature by rt AND mz
matched_peak_neg = matched_peak_area_df_neg.sort_values(by=['Average Rt(min)', 'Average Mz'])

#filerted features with intensity lower than 5000 
peak_areaneg = matched_peak_neg[[col for col in matched_peak_neg.columns if col.startswith('BH')]]
row_mediansneg = peak_areaneg.median(axis=1)
peak_indexneg = row_mediansneg[row_mediansneg >= 0].index
filter_resultsneg = matched_peak_neg.loc[peak_indexneg]
print(filter_resultsneg.shape)
# filter_resultsneg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/matched_peak_neg_over5000_wopredRT.csv")

# Step 4: Retain columns with median values > 5000, filter non-numeric values, and perform DataFrame transpose
# filtered_df = matched_peak_area_df_neg.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
# filtered_df = filtered_df.loc[:, (filtered_df.median(axis=0) >= 5000)].dropna(how='all', axis=1)  # Retain columns with median value > 5000

Matching compounds: 100%|██████████| 5267/5267 [01:33<00:00, 56.58it/s]


Number of matched unique feature from peak area table: 6610
Number of matched unique chemicals from peak area table: 2122
#of targetedmz with hits:(2122, 15)
(14606, 226)


In [ ]:
# filter_resultsneg = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/matched_peak_neg_over5000_wopredRT.csv")

#feature prioritization and further annotation and confirmation
#annotate the features according to RT_library and predictedRT
RTmatch_threshold = 0.6 #RT match threshold in minutes
#convert RT_library to numeric values
filter_resultsneg['RT_library'] = pd.to_numeric(filter_resultsneg['RT_library'], errors='coerce')

for iter, row in filter_resultsneg.iterrows():
    if (pd.notnull(row['RT_library'])) and ((row['RT_polarity'] == 'both') or (row['RT_polarity'] == 'neg')):
        if (abs(row['Average Rt(min)'] - row['RT_library']) <= RTmatch_threshold):
            filter_resultsneg.loc[iter, 'annotation_RTMS'] = '2'
        else:
            filter_resultsneg.loc[iter, 'annotation_RTMS'] = 'NotMatched'
    elif (row['RT_polarity'] =='na') or (row['RT_polarity'] == 'pos'):
        filter_resultsneg.loc[iter, 'annotation_RTMS'] = 'NotMatched'
    else:
        if (row['Average Rt(min)'] >= row['PredictedRT_lower']) & (row['Average Rt(min)'] <= row['PredictedRT_upper']):
            filter_resultsneg.loc[iter, 'annotation_RTMS'] = '3'
        else:
            filter_resultsneg.loc[iter, 'annotation_RTMS'] = '4'
            
print(filter_resultsneg[filter_resultsneg['annotation_RTMS'] == '2'].shape)
print(filter_resultsneg[filter_resultsneg['annotation_RTMS'] == '3'].shape)

#output the datafile for spectrum matching
#filter_resultsneg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/feature_annotation_RTMS_neg_peakover5000_20241228.csv")
#forward to the next step for MSMS insilico or experimental matching and validation

(43, 227)
(10216, 227)


In [30]:
print(filter_resultsneg[filter_resultsneg['annotation_RTMS'] == 'NotMatched'].shape)
print(filter_resultspos[filter_resultspos['annotation_RTMS'] == 'NotMatched'].shape)

(308, 230)
(273, 229)


In [26]:
##check hits with level 2
level2_pos = filter_resultspos[filter_resultspos['annotation_RTMS'] == '2']
level2_neg = filter_resultsneg[filter_resultsneg['annotation_RTMS'] == '2']
print(level2_pos.shape)
print(level2_neg.shape)

(53, 229)
(43, 230)


##retreive msms injection patch for library matching

In [ ]:
#acquired msms data 
neg_msmspath1 = "D:/UCSF_postdoc_topic/REVEAL_topics/R200_MSMS/targeted_msms_check_spectrumsearch_neg.csv"
pos_msmspath1 = "D:/UCSF_postdoc_topic/REVEAL_topics/R200_MSMS/targeted_msms_check_spectrumsearch_pos.csv"
pos_msmspath2 = "D:/UCSF_postdoc_topic/REVEAL_topics/R200_MSMS/B1-10pool_tMSMS_04012025/feature_annotation_RTMS_pos_allfeaturesMplusH_20250401_30permin_formsinjection.csv"
neg_msmspath2 = "D:/UCSF_postdoc_topic/REVEAL_topics/R200_MSMS/B1-10pool_tMSMS_04012025/feature_annotation_RTMS_neg_allfeaturesMplusH_20250401_50permin_formsinjection.csv"
neg_msms = pd.read_csv(neg_msmspath1)
pos_msms = pd.read_csv(pos_msmspath1)
neg_msms2 = pd.read_csv(neg_msmspath2)
pos_msms2 = pd.read_csv(pos_msmspath2)

#combined averageRT and averageMz to create the new column feature_id for neg_msms and pos_msms
neg_msms['feature_id'] = neg_msms['Average Rt(min)'].astype(str) + '_' + neg_msms['Average Mz'].astype(str)
pos_msms['feature_id'] = pos_msms['Average Rt(min)'].astype(str) + '_' + pos_msms['Average Mz'].astype(str)

neg_msms2['feature_id'] = neg_msms2['Average Rt(min)'].astype(str) + '_' + neg_msms2['Average Mz'].astype(str)
pos_msms2['feature_id'] = pos_msms2['Average Rt(min)'].astype(str) + '_' + pos_msms2['Average Mz'].astype(str)
#output the datafile for spectrum matching

#match the targeted msms list with the new feature list 
for iter, row in filter_resultsneg.iterrows():
    matches = neg_msms[neg_msms['feature_id'] == row['feature_id']]
    matches2 = neg_msms2[neg_msms2['feature_id'] == row['feature_id']]
    if not matches.empty:
        filter_resultsneg.loc[iter, 'MSMS_injection'] = matches['MSMS_injection'].values[0]
    else:
        filter_resultsneg.loc[iter, 'MSMS_injection'] = 'NotMatched'
    if not matches2.empty:
        filter_resultsneg.loc[iter, 'MSMS_injection2'] = matches2['injection'].values[0]
    else:
        filter_resultsneg.loc[iter, 'MSMS_injection2'] = 'NotMatched'

for iter, row in filter_resultspos.iterrows():
    matches = pos_msms[pos_msms['feature_id'] == row['feature_id']]
    matches2 = pos_msms2[pos_msms2['feature_id'] == row['feature_id']]
    if not matches.empty:
        filter_resultspos.loc[iter, 'MSMS_injection'] = matches['MSMS_injection'].values[0]
    else:
        filter_resultspos.loc[iter, 'MSMS_injection'] = 'NotMatched'
    if not matches2.empty:
        filter_resultspos.loc[iter, 'MSMS_injection2'] = matches2['injection'].values[0]

    else:
        filter_resultspos.loc[iter, 'MSMS_injection2'] = 'NotMatched'

#output the datafile for spectrum matching
# filter_resultspos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/feature_annotation_RTMS_pos_MSMSinjection.csv")
# filter_resultsneg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/feature_annotation_RTMS_neg_MSMSinjection.csv")

C:\Users\yangj\AppData\Local\Temp\ipykernel_26968\137290428.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NotMatched' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  filter_resultsneg.loc[iter, 'MSMS_injection2'] = 'NotMatched'
C:\Users\yangj\AppData\Local\Temp\ipykernel_26968\137290428.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NotMatched' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  filter_resultspos.loc[iter, 'MSMS_injection2'] = 'NotMatched'


In [ ]:
filter_resultspos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/feature_annotation_RTMS_pos_MSMSinjection.csv") #(12058,229)
filter_resultsneg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/feature_annotation_RTMS_neg_MSMSinjection.csv") #(14606,230)

In [ ]:
targetmz_path = ["D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/plastic_map_chemlist/Plastic_Chemical_withRT_CFMID_spectrumdatabase_chemical_annotation_20250404.csv"] #plastic chemical list with predicted/library RT, CRMID, and spectrum ID match record,
